# Парсим TEI с помощью lxml

lxml - это питоновский модуль для работы с XML и HTML: с его помощью можно как создавать, так и парсить XML/HTML. В случае с TEI нас интересует XML. Давайте попробуем распарсить малый академический словарь (лежит в этом же репозитории, *MAS.tei*).

## etree

Для парсинга xml используется etree. Импортируем его и считаем им файл в tei.

In [1]:
from lxml import etree

In [3]:
# открываем файл со словарём
with open('MAS.tei') as f:
    text = f.read()
print(text[:200])

<xml>
	<fileDesc>
		<respStmt>
			<name>Вячеслав Иванов</name>
		</respStmt>
		<extent>1607</extent>
		<sourceDesc>
			<ref target="ссылка на источник в интернете">http://feb-web.ru/feb/mas/mas-abc/de


In [4]:
# создаём дерево etree из строки
root = etree.fromstring(text)

In [5]:
root.tag

'xml'

In [7]:
root[0][0].tag

'respStmt'

In [9]:
print(root[0][0][0].tag)
print(root[0][0][0].text)

name
Вячеслав Иванов


## XPath

! Вся описательная часть этого раздела скопирована из [туториала Эли Мустакимовой](https://github.com/elmiram/2017learnpython/blob/master/2%20%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%20-%20lxml.ipynb). Спасибо Эле!

XPath - это способ представления пути к информации в структурированных документах типа XML/HTML. В XPath используются специальные выражения для того, чтобы выбирать какие-то узлы (элементы) в дереве XML. Есть туториал по XPath на w3schools, очень советую его почитать.

Элементы XPath выражения:
* tag - выбрать все узлы с названием tag
* / - указывает на узлы, которые являются прямыми потомками текущего
* // - указывает на узлы, которые являются потомками текущего (где-то внутри текущего узла, но необязательно его потомок)
* . - текущий узел
* .. - родитель текущего узла
* @ - указывает на атрибут
* text() - выбирает весь текст внутри узла (и внутри дочерних узлов тоже)

В XPath выражении могут быть предикаты, которые позволяют выбрать конкретный узел, предикаты пишутся в квадратных скобках. Например:
* //p[1] - выбрать первый элемент, который является ребенком узла с тэгом p
* //p[@style] - выбрать все узлы с тэгом p, у которых есть атрибут style
* //td[@border="1"] - выбрать все узлы с тэгом td, у которых атрибут border равен 1

Также есть специальные знаки, означающие "любой":
* \* - любой узел (например //\* - все элементы дерева, /body/\* - все дети тэга body)
* @ - любой атрибут (например `//p[@]` - все узлы с тэгом p, у которых есть хоть один какой-то атрибут)

Например, если мы хотим найти все даты на нашей странице, мы соберем такое выражение:

> //span[@class="post__time_published"]/text()

Все заголовки:
> //a[@class="post__title_link"]/text()

### А тепеть давайте попробуем применить это к нашим данным: